# LangChain API Workflow

Based on our problem statement, we need to build a customer service chatbot. From a technical requirments standpoint, we need

- A generative AI chatbot with access to our internal documentation

- A generative AI chatbot that is stateful (ie: chat history)

## Load Libraries

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_community.retrievers import AzureAISearchRetriever

## Create Prompt

We are going to leverage the [MessagesPlaceholder](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.MessagesPlaceholder.html) prompt to help manage the chat history.

In [2]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

## Initialize LLM Model

In [3]:
model = AzureChatOpenAI(
    azure_deployment="gpt4o",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01"
)

## Initialize Retriever

In [4]:
retriever = AzureAISearchRetriever(
    content_key="content", top_k=5, index_name="product-info", api_key=os.getenv("AZURE_SEARCH_KEY"), service_name="genai-on-azure-search"
)

## Initialize the Chain

In [5]:
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

In [6]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [8]:
conversational_rag_chain.invoke(
    {"input": "How much is the home theater system?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run c921f263-a3b7-4ce7-af4b-15c14e2d6f07 not found for run ec970d9f-b752-4a09-b3a9-d5d29d30978d. Treating as a root run.


'The Ultimate Home Theater System (UHTS-1000) is priced at $4999.'

In [9]:
conversational_rag_chain.invoke(
    {"input": "What does it include?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

Parent run e6018b6f-a27d-4309-8712-0f9520a5af72 not found for run bfdac4fb-bc46-4d86-89ef-ac6c4a305de1. Treating as a root run.


'The Ultimate Home Theater System (UHTS-1000) includes the following:\n\n- 75-inch 8K Ultra HD Smart TV\n- 60-inch soundbar with integrated upward-firing speakers\n- 12-inch wireless subwoofer\n- Four wireless rear speakers with integrated height speakers\n- Remote control for TV\n- Remote control for sound system\n- Wall mount bracket for TV\n- User manual and quick start guide\n- HDMI cables (3)\n- Power cables\n- Batteries for remote controls'

In [10]:
from langchain_core.messages import AIMessage

for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: How much is the home theater system?

AI: The Ultimate Home Theater System (UHTS-1000) is priced at $4999.

User: What does it include?

AI: The Ultimate Home Theater System (UHTS-1000) includes the following:

- 75-inch 8K Ultra HD Smart TV
- 60-inch soundbar with integrated upward-firing speakers
- 12-inch wireless subwoofer
- Four wireless rear speakers with integrated height speakers
- Remote control for TV
- Remote control for sound system
- Wall mount bracket for TV
- User manual and quick start guide
- HDMI cables (3)
- Power cables
- Batteries for remote controls

